## Plot Results

This notebook calls cluster_analysis.py, performs nearest-point interpolation to determine the 5 zones, and plots on a map of Madrid. Annotations of each block of code are below them. Annotation of cluster_analysis can be found in cluster_analysis.ipymb.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
from scipy import interpolate

Frequent user of matplotlib and the various scipy interpolate functions. First time using Basemap - recommended by a colleague as it's widely used within our department and well documented. 

In [ ]:
import cluster_analysis
cluster_analysis.cluster_analysis()

Import and call cluster_analysis from cluster_analysis.py

In [ ]:
min_lon = -3.93455628
min_lat = 40.25387182
max_lon = -3.31993445
max_lat = 40.57085727
lats=np.arange(min_lat,max_lat,0.001)
lons=np.arange(min_lon,max_lon,0.001)
lons_grid, lats_grid = np.meshgrid(lons,lats)

Setting up the grid at 0.001 degree resolution. 

In [ ]:
ref_grid = np.zeros((np.size(lats_grid), 2))
ref_grid[:, 0] = np.ravel(lats_grid)
ref_grid[:, 1] = np.ravel(lons_grid)
pointdata=pd.read_csv('madrid_poi_clusters.csv')
values = np.array(pointdata['cluster_id'])
points = np.array([pointdata['lat'],pointdata['lon']])
data = interpolate.griddata(points.T, values, ref_grid, method = 'nearest')
data=np.reshape(data,np.shape(lats_grid))
data = data.astype('float32')

I decided to present this data on a map and determine the 5 zones by using nearest point interpolation. I used scipy's griddata function as it takes ungridded data as the reference, in this case the POIs, and returns it to a regular grid, i.e the one created by the exercise's given coordinates. 

In [ ]:
map = Basemap(llcrnrlon=min_lon,llcrnrlat=min_lat,urcrnrlon=max_lon,urcrnrlat=max_lat, resolution = 'h', epsg=2062)
map.arcgisimage(service='World_Shaded_Relief', xpixels = 3000, verbose= True, zorder=1)
map.readshapefile('/home/Earth/mfalls/Downloads/gadm36_ESP_4', '')

Created the Basemap object to use in the plot. To present it as a map, I considered satellite view but it made the map look too cluttered and it distracted from the results. In addition, I used the municipal boundaries obtained from https://gadm.org/download_country.html. 

In [ ]:
map_min_lon,map_min_lat = map(min_lon, min_lat)
map_max_lon,map_max_lat = map(max_lon, max_lat)
x = np.linspace(0, map.urcrnrx, data.shape[1])  
y = np.linspace(0, map.urcrnry, data.shape[0])
xx, yy = np.meshgrid(x, y)

Setting up the grid for plotting. I found this a bit tricky trying to understand the relationship between map() and the coordinates. I'm used to having (lat,lon) convention rather than (lon,lat). 

In [ ]:
px,py = map(np.array(pointdata['lon']), np.array(pointdata['lat']))
map.scatter(px, py, marker='.', color='r',s=0.1,zorder=3)
map.pcolormesh(xx, yy, data,zorder=2,alpha=0.2)
plt.show()
plt.savefig('madrid_poi_clusters.png')

Plotting the 5 zones on the map, and also all of the POIs.